In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import fitz 

url = 'https://www.railpassengers.org/resources/ridership-statistics/'
url_abe = 'https://www.railpassengers.org/site/assets/files/1679/abe.pdf'
page = requests.get(url_abe)
page

<Response [200]>

In [2]:
# Save the PDF file locally
pdf_path = 'abe.pdf'
with open(pdf_path, 'wb') as file:
    file.write(page.content)
    
# Step 2: Extract text from the PDF file
def extract_text_from_pdf(pdf_path):
    # Open the PDF file
    document = fitz.open(pdf_path)
    
    # Iterate through each page and extract text
    text = ''
    for page_num in range(len(document)):
        page = document.load_page(page_num)
        text += page.get_text()
    
    return text

# Get the extracted text
pdf_text = extract_text_from_pdf(pdf_path)

#### Grab all the PDF links

In [3]:
import requests
from bs4 import BeautifulSoup

# URL of the webpage to scrape
url = "https://www.railpassengers.org/resources/ridership-statistics/"

# Send a GET request to the URL
response = requests.get(url)

# Check response
print(response)

<Response [200]>


In [4]:
import requests
from bs4 import BeautifulSoup

# URL of the webpage to scrape
url = "https://www.railpassengers.org/resources/ridership-statistics/"

# Send a GET request to the URL
response = requests.get(url)
soup = BeautifulSoup(response.content, "html.parser")

# Find all option elements and extract their values
option_values = [option['value'] for option in soup.find_all('option') if 'value' in option.attrs]

# Print the list of option values
print(option_values)


['', '/site/assets/files/1174/al.pdf', '/site/assets/files/1176/az.pdf', '/site/assets/files/1177/ar.pdf', '/site/assets/files/1178/ca.pdf', '/site/assets/files/1179/co.pdf', '/site/assets/files/1180/ct.pdf', '/site/assets/files/1181/de.pdf', '/site/assets/files/1182/fl.pdf', '/site/assets/files/1183/ga.pdf', '/site/assets/files/1185/id.pdf', '/site/assets/files/1186/il.pdf', '/site/assets/files/1187/in.pdf', '/site/assets/files/1188/ia.pdf', '/site/assets/files/1189/ks.pdf', '/site/assets/files/1190/ky.pdf', '/site/assets/files/1191/la.pdf', '/site/assets/files/1192/me.pdf', '/site/assets/files/1193/md.pdf', '/site/assets/files/1194/ma.pdf', '/site/assets/files/1195/mi.pdf', '/site/assets/files/1196/mn.pdf', '/site/assets/files/1197/ms.pdf', '/site/assets/files/1198/mo.pdf', '/site/assets/files/1199/mt.pdf', '/site/assets/files/1200/ne.pdf', '/site/assets/files/1201/nv.pdf', '/site/assets/files/1202/nh.pdf', '/site/assets/files/1203/nj.pdf', '/site/assets/files/1204/nm.pdf', '/site/as

In [71]:
option_values = option_values[1:len(option_values)]
len(option_values)

1090

## Only extract the URLs with XXX.pdf format

In [73]:
import requests
from bs4 import BeautifulSoup

# URL of the webpage to scrape
url = "https://www.railpassengers.org/resources/ridership-statistics/"

# Send a GET request to the URL
response = requests.get(url)
soup = BeautifulSoup(response.content, "html.parser")

# Initialize an empty list to store the option values
option_values = []

# Find all option elements
options = soup.find_all('option')

# Iterate over each option element and extract its value
for option in options:
    if 'value' in option.attrs:
        value = option['value']
        # Extract the part after the last slash and before .pdf
        part_after_last_slash = value.split('/')[-1]
        filename = part_after_last_slash.split('.')[0]
        # Check if the part has more than two characters
        if len(filename) > 2 and value.endswith('.pdf'):
            option_values.append(value)

In [74]:
len(option_values)

989

## Remove any URLs that have pattern: XX##.pdf

In [84]:
import requests
from bs4 import BeautifulSoup
import re

# URL of the webpage to scrape
url = "https://www.railpassengers.org/resources/ridership-statistics/"

# Send a GET request to the URL
response = requests.get(url)
soup = BeautifulSoup(response.content, "html.parser")

# Initialize an empty list to store the option values
option_values = []

# Find all option elements
options = soup.find_all('option')

# Regular expression pattern to match filenames ending in two letters and two numbers before .pdf
pattern = re.compile(r'[A-Za-z]{2}\d{2}\.pdf$')

# Iterate over each option element and extract its value
for option in options:
    if 'value' in option.attrs:
        value = option['value']
        # Check if the value matches the undesired pattern
        if len(value.split('/')[-1].split('.')[0]) > 2 and value.endswith('.pdf') and not pattern.search(value):
            option_values.append(value)

In [85]:
len(option_values)

553

In [86]:
option_values.remove("/site/assets/files/6872/thruway.pdf")
len(option_values)

552

### Proof of Concept: Pull out the ridership numbers

In [77]:
import requests
import pandas as pd
import fitz
import re

# URL of the PDF file
url_abe = 'https://www.railpassengers.org/site/assets/files/1679/abe.pdf'

# Step 1: Download the PDF file
page = requests.get(url_abe)

# Save the PDF file locally
pdf_path = 'abe.pdf'
with open(pdf_path, 'wb') as file:
    file.write(page.content)

# Step 2: Extract text from the PDF file
def extract_text_from_pdf(pdf_path):
    # Open the PDF file
    document = fitz.open(pdf_path)
    
    # Iterate through each page and extract text
    text = ''
    for page_num in range(len(document)):
        page = document.load_page(page_num)
        text += page.get_text()
    
    return text

# Get the extracted text
pdf_text = extract_text_from_pdf(pdf_path)

# Step 3: Parse the text to extract all numbers in the XX,XXX format
def extract_all_ridership_numbers(text):
    # Define a regex pattern to find all numbers formatted like "XX,XXX"
    pattern = re.compile(r'\b\d{2},\d{3}\b')
    
    # Find all matches in the text
    matches = pattern.findall(text)
    
    # Return all matches
    return matches

# Extract all ridership numbers
ridership_numbers = extract_all_ridership_numbers(pdf_text)

df = pd.DataFrame(ridership_numbers, columns=['Ridership Numbers'])[0:7]
df['Year'] = [2016,2017,2018,2019,2020,2021,2022]
df['Station'] = 'Test'
df = df.pivot(index = 'Station', columns="Year", values="Ridership Numbers").reset_index()

df

Year,Station,2016,2017,2018,2019,2020,2021,2022
0,Test,"37,161","37,045","37,867","39,108","19,743","14,180","34,040"


### Loop through all of the URL chunks

In [89]:
option_values[2]

'/site/assets/files/1685/alb.pdf'

In [94]:
import requests
from bs4 import BeautifulSoup

# URL of the webpage to scrape
url = "https://www.railpassengers.org/resources/ridership-statistics/"


end = len(option_values)
list_of_url_chunks = option_values
url = 'https://www.railpassengers.org'


ridership_data = []

for i in list_of_url_chunks:
        # Step 1: Download the PDF file
        page = requests.get(url + i)

        # Save the PDF file locally
        pdf_path = i.rsplit('/', 1)[1]
        with open(pdf_path, 'wb') as file:
            file.write(page.content)

        # Step 2: Extract text from the PDF file
        def extract_text_from_pdf(pdf_path):
            # Open the PDF file
            document = fitz.open(pdf_path)

            # Iterate through each page and extract text
            text = ''
            for page_num in range(len(document)):
                page = document.load_page(page_num)
                text += page.get_text()

            return text

        # Get the extracted text
        pdf_text = extract_text_from_pdf(pdf_path)

        # Step 3: Parse the text to extract all numbers in the XX,XXX format
        def extract_all_ridership_numbers(text):
            # Define a regex pattern to find all numbers formatted like "XX,XXX"
            pattern = re.compile(r'\b\d{2},\d{3}\b')

            # Find all matches in the text
            matches = pattern.findall(text)

            # Return all matches
            return matches

        # Extract all ridership numbers
        ridership_numbers = extract_all_ridership_numbers(pdf_text)

        # Check where we're at...
        print('URL link:', i)


        df = pd.DataFrame(ridership_numbers, columns=['Ridership Numbers'])#[0:7]
        #df['Year'] = [2016,2017,2018,2019,2020,2021,2022]
        df['Station'] = i
        #df = df.pivot(index = 'Station', columns="Year", values="Ridership Numbers").reset_index()
        ridership_data.append(df)


URL link: /site/assets/files/1679/abe.pdf
URL link: /site/assets/files/1684/aly.pdf
URL link: /site/assets/files/1685/alb.pdf
URL link: /site/assets/files/1686/ali.pdf
URL link: /site/assets/files/1687/abq.pdf
URL link: /site/assets/files/1689/ald.pdf
URL link: /site/assets/files/1690/alx.pdf
URL link: /site/assets/files/1691/alc.pdf
URL link: /site/assets/files/1693/alp.pdf
URL link: /site/assets/files/1694/aln.pdf
URL link: /site/assets/files/1695/alt.pdf
URL link: /site/assets/files/5017/amm.pdf
URL link: /site/assets/files/1696/ams.pdf
URL link: /site/assets/files/1697/ana.pdf
URL link: /site/assets/files/1698/arb.pdf
URL link: /site/assets/files/1699/atn.pdf
URL link: /site/assets/files/1700/aca.pdf
URL link: /site/assets/files/6876/acd.pdf
URL link: /site/assets/files/1704/ard.pdf
URL link: /site/assets/files/1703/adm.pdf
URL link: /site/assets/files/1705/ark.pdf
URL link: /site/assets/files/1708/asd.pdf
URL link: /site/assets/files/1706/aky.pdf
URL link: /site/assets/files/1712/

URL link: /site/assets/files/5011/guf.pdf
URL link: /site/assets/files/5010/gut.pdf
URL link: /site/assets/files/2011/ham.pdf
URL link: /site/assets/files/2012/hmd.pdf
URL link: /site/assets/files/2013/hmi.pdf
URL link: /site/assets/files/2017/hnf.pdf
URL link: /site/assets/files/2018/hfy.pdf
URL link: /site/assets/files/2019/har.pdf
URL link: /site/assets/files/2020/hfd.pdf
URL link: /site/assets/files/2021/has.pdf
URL link: /site/assets/files/2023/hbg.pdf
URL link: /site/assets/files/2026/hhl.pdf
URL link: /site/assets/files/2027/hav.pdf
URL link: /site/assets/files/2028/hay.pdf
URL link: /site/assets/files/2029/haz.pdf
URL link: /site/assets/files/2031/her.pdf
URL link: /site/assets/files/2034/hem.pdf
URL link: /site/assets/files/2035/hpt.pdf
URL link: /site/assets/files/2037/hin.pdf
URL link: /site/assets/files/2038/hld.pdf
URL link: /site/assets/files/2039/hom.pdf
URL link: /site/assets/files/2040/hol.pdf
URL link: /site/assets/files/5009/hlk.pdf
URL link: /site/assets/files/2041/

URL link: /site/assets/files/2364/prc.pdf
URL link: /site/assets/files/2365/pct.pdf
URL link: /site/assets/files/2366/pjc.pdf
URL link: /site/assets/files/2367/pvd.pdf
URL link: /site/assets/files/2368/pro.pdf
URL link: /site/assets/files/2372/pur.pdf
URL link: /site/assets/files/2374/qan.pdf
URL link: /site/assets/files/2375/qcy.pdf
URL link: /site/assets/files/2379/rgh.pdf
URL link: /site/assets/files/2380/rph.pdf
URL link: /site/assets/files/2381/rtl.pdf
URL link: /site/assets/files/2382/rat.pdf
URL link: /site/assets/files/2384/rdw.pdf
URL link: /site/assets/files/2385/rdd.pdf
URL link: /site/assets/files/2390/rno.pdf
URL link: /site/assets/files/2391/ren.pdf
URL link: /site/assets/files/2392/rhi.pdf
URL link: /site/assets/files/2394/ric.pdf
URL link: /site/assets/files/2395/rvm.pdf
URL link: /site/assets/files/2396/rvr.pdf
URL link: /site/assets/files/2400/riv.pdf
URL link: /site/assets/files/2401/rnk.pdf
URL link: /site/assets/files/2402/roc.pdf
URL link: /site/assets/files/2407/

In [95]:
ridership_data

[  Ridership Numbers                          Station
 0            37,161  /site/assets/files/1679/abe.pdf
 1            37,045  /site/assets/files/1679/abe.pdf
 2            37,867  /site/assets/files/1679/abe.pdf
 3            39,108  /site/assets/files/1679/abe.pdf
 4            19,743  /site/assets/files/1679/abe.pdf
 5            14,180  /site/assets/files/1679/abe.pdf
 6            34,040  /site/assets/files/1679/abe.pdf
 7            34,040  /site/assets/files/1679/abe.pdf
 8            34,040  /site/assets/files/1679/abe.pdf
 9            32,991  /site/assets/files/1679/abe.pdf,
    Ridership Numbers                          Station
 0             33,656  /site/assets/files/1684/aly.pdf
 1             33,933  /site/assets/files/1684/aly.pdf
 2             30,780  /site/assets/files/1684/aly.pdf
 3             32,856  /site/assets/files/1684/aly.pdf
 4             19,082  /site/assets/files/1684/aly.pdf
 5             13,959  /site/assets/files/1684/aly.pdf
 6             26,93

In [97]:
type(ridership_data)

list

In [100]:
pre_cleaned_ridership_data = pd.concat(ridership_data)
pre_cleaned_ridership_data.shape

(2340, 2)

In [102]:
pre_cleaned_ridership_data.to_csv('pcrd.csv')